In [ ]:
! pip install plotnine

In [ ]:
import pandas as pd
from plotnine import *
from sklearn.manifold import TSNE
# from sklearn.decomposition import PCA, TruncatedSVD
from util.datasource import anything, corroles
from util.analysis import ext_modes, plot_selector, perc_comp, perc_ext


In [ ]:
df = corroles.dataFrame

In [ ]:
conformation = df[perc_ext].fillna(0)
# conformation["Doop"] = df["Doop (exp.)"]
# conformation["Cavity"] = df["Cavity"]

In [ ]:
tsne = TSNE(n_components=2)

In [ ]:
X = conformation.values

In [ ]:
mapping_tsne = tsne.fit_transform(X)

In [ ]:
df_tsne = pd.DataFrame(mapping_tsne, columns=["x", "y"])

In [ ]:
df_tsne["category"] = df["category"]
df_tsne["CCDC"] = df["CCDC"]
df_tsne["M"] = df["M"]
df_tsne["Group"] = df["Group"]
df_tsne["Doop"] = df["Doop (exp.)"]
df_tsne["Cavity"] = df["Cavity"]
df_tsne = df_tsne.astype({"Group": "str"})
df_tsne


In [ ]:
ggplot(df_tsne, aes(x="x", y="y")) + geom_point(aes(label="M", colour="Group", size="Doop")) + geom_text(aes(label="M"), size=6)
